In [1]:
synthetic_data_path = "C:/Users/solom/Desktop/AliceBob25/github/eth25_alice-bob_challenge/data/synthetic"
experimental_data_path = "C:/Users/solom/Desktop/AliceBob25/github/eth25_alice-bob_challenge/data/experimental"

import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy
import sklearn
import jax.numpy as jnp
import dynamiqs as dq
from scipy.ndimage import gaussian_filter
from skimage.metrics import structural_similarity as ssim
import cvxpy as cp

from matplotlib.animation import FuncAnimation
from PIL import Image
import imageio
import os

results_path = "C:/Users/solom/Desktop/AliceBob25/"

In [2]:
for file in os.listdir(experimental_data_path):
    if file.endswith(".pickle"):
        print(f"Processing {file}...")
        with open(os.path.join(experimental_data_path, file), "rb") as f:
            data = pickle.load(f)
        # Extract the relevant data
        xvec, yvec, W = data
        # Create a figure and axis
        fig, ax = plt.subplots()
        # Create a meshgrid for the x and y vectors
        X, Y = np.meshgrid(xvec, yvec, indexing='ij')
        # Plot the data using pcolormesh
        c = ax.pcolormesh(X, Y, W, shading='nearest')
        # Add a colorbar
        fig.colorbar(c, ax=ax)
        # Set the title and labels
        ax.set_title(file)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        # Save the figure
        results_path = "C:/Users/solom/Desktop/AliceBob25/results_experimental_data/wigner_original/"
        fig.savefig(results_path + str(file.split('.')[0]) + ".png", bbox_inches='tight', dpi=300)
        plt.close()



Processing wigner_cat_minus.pickle...
Processing wigner_cat_plus.pickle...
Processing wigner_cat_plus_113.pickle...
Processing wigner_cat_plus_25.pickle...
Processing wigner_cat_plus_56.pickle...
Processing wigner_fock_one.pickle...
Processing wigner_fock_zero.pickle...


In [3]:
# Inverse the affine transform

def estimate_b_robust(W, edge_width=10):
    """Estimate background offset b using only finite values on the edges."""
    edges = np.concatenate([
        W[:edge_width, :].ravel(),
        W[-edge_width:, :].ravel(),
        W[:, :edge_width].ravel(),
        W[:, -edge_width:].ravel()
    ])
    finite_edges = edges[np.isfinite(edges)]  # remove NaN, inf
    return np.mean(finite_edges)

def correct_affine_wigner(W, xvec, yvec, edge_width=10):
    """
    Corrects affine distortion in a Wigner function.

    Parameters:
    - W: 2D numpy array (Wigner function)
    - xvec, yvec: 1D arrays defining the phase space grid
    - edge_width: width of the boundary region for estimating b

    Returns:
    - W_corrected: 2D corrected Wigner function
    - a: normalization factor
    - b: background offset
    """
    dx = xvec[1] - xvec[0]
    dp = yvec[1] - yvec[0]

    # Estimate b from edge values
    b = estimate_b_robust(W, edge_width)

    # Estimate a from integral of (W - b)
    W_masked = np.where(np.isfinite(W), W, 0.0)  # zero out NaNs
    a = np.sum(W_masked - b) * dx * dp
    a = np.abs(a)  # ensure a is positive
    W_corrected = (W_masked - b) / a

    return W_corrected, a, b


In [11]:
name_list = ["wigner_cat_minus",
            "wigner_cat_plus",
            "wigner_cat_plus_113",
            "wigner_cat_plus_25",
            "wigner_cat_plus_56",
            "wigner_fock_one",
            "wigner_fock_zero"]

for name in name_list:
    path = experimental_data_path + "/" + name + ".pickle"
    with open(path, "rb") as f:
        data = pickle.load(f)
    # Extract the relevant data
    xvec, yvec, W = data
    print(xvec.shape, yvec.shape, W.shape)
    W, a, b = correct_affine_wigner(W, xvec, yvec)
    np.savez(f"{name}_affine_corrected.npz", Wigner=W, x=xvec, y=yvec)
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Create a meshgrid for the x and y vectors
    X, Y = np.meshgrid(xvec, yvec, indexing='ij')
    # Plot the data using pcolormesh
    c = ax.pcolormesh(X, Y, W, shading='nearest')
    # Add a colorbar
    fig.colorbar(c, ax=ax)
    fig.text(0.05, 0.95, f"a: {a:.2f}\n b: {b:.2f}", transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
    # Set the title and labels
    ax.set_title(name)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    # Save the figure
    results_path = "C:/Users/solom/Desktop/AliceBob25/results_experimental_data/affine_correction/"
    fig.savefig(results_path + str(name) + ".png", bbox_inches='tight', dpi=300)
    plt.close()


(250,) (100,) (250, 100)
(250,) (100,) (250, 100)
(125,) (38,) (125, 38)
(250,) (77,) (250, 77)
(250,) (77,) (250, 77)
(100,) (100,) (100, 100)
(100,) (100,) (100, 100)


In [7]:
#Try some smoothing
def smooth_wigner(W, sigma):
    """
    Smooth the Wigner function using a Gaussian filter.

    Parameters:
    - W: 2D numpy array (Wigner function)
    - sigma: standard deviation for Gaussian kernel

    Returns:
    - W_smoothed: 2D smoothed Wigner function
    """
    return gaussian_filter(W, sigma=sigma)

sigma = np.linspace(0.1, 5, 10)

for name in name_list:
    path = experimental_data_path + "/" + name + ".pickle"
    with open(path, "rb") as f:
        data = pickle.load(f)
    # Extract the relevant data
    xvec, yvec, W = data
    W, a, b = correct_affine_wigner(W, xvec, yvec)
    for s in sigma:
        W_smooth = smooth_wigner(W, s)
        # Save the smoothed Wigner function
        results_path = "C:/Users/solom/Desktop/AliceBob25/results_experimental_data/smoothened_data/"
        np.savez(results_path + f"{name}_affine_corrected_smooth_sigma_{s}.npz", Wigner=W_smooth, x=xvec, y=yvec)
        # Create a figure and axis
        fig, ax = plt.subplots()
        # Create a meshgrid for the x and y vectors
        X, Y = np.meshgrid(xvec, yvec, indexing='ij')
        # Plot the data using pcolormesh
        c = ax.pcolormesh(X, Y, W_smooth, shading='nearest')
        # Add a colorbar
        fig.colorbar(c, ax=ax)
        # Set the title and labels
        ax.set_title(name + f" sigma: {s:.2f}")
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        # Save the figure
        results_path = "C:/Users/solom/Desktop/AliceBob25/results_experimental_data/smoothened_images/"
        fig.savefig(results_path + str(name) + "_sigma_" + str(s) + ".png", bbox_inches='tight', dpi=300)
        plt.close()


In [10]:
sigma = np.linspace(0.1, 8, 10)
for name in name_list:
    data = pickle.load(open(experimental_data_path + "/" + name + ".pickle", "rb"))
    xvec, yvec, W = data
    W, a, b = correct_affine_wigner(W, xvec, yvec)

    # Create a single figure for all sigmas for this file
    fig, ax = plt.subplots(5, 2, figsize=(10, 20))
    fig.subplots_adjust(hspace=0.5)

    # Meshgrid once
    X, Y = np.meshgrid(xvec, yvec, indexing='ij')

    for i, s in enumerate(sigma):
        W_smooth = smooth_wigner(W, s)
        c = ax[i//2, i%2].pcolormesh(X, Y, W_smooth, shading='nearest')
        fig.colorbar(c, ax=ax[i//2, i%2])
        ax[i//2, i%2].set_title(f"{name}, σ={s:.2f}")
        ax[i//2, i%2].set_xlabel('x')
        ax[i//2, i%2].set_ylabel('y')

    # Save the full figure
    results_path = "C:/Users/solom/Desktop/AliceBob25/results_experimental_data/smoothened_images/"
    fig.savefig(results_path + f"{name}_scanned.png", bbox_inches='tight', dpi=300)
    plt.close()


| Experimental Data | Ideal Smoothing σ (Approximated) |
|-------------------|-------------|
| cat_minus            | 3.0         |
| cat_plus_25            | 2.4         |
| cat_plus_56            | 2.7         |
| cat_plus_113            | 1.4         |
| cat_plus            | 2.7         |
| fock_one            | 3.0         |
| fock_zero            | 3.6         |

The experimental data behaves from what we have seen considerably better compared to the synthetic data when applying the gaussian filtering. When reversing the affine transformation of the Wigner function, we notice that the values of b are lower than the ones in the synthetic images. We might draw the conclusion that the additive scaling parameter is not a very relevant effect of real life noise sources. However, gaussian noise and scaling parameter a are impacting the Wigner's function. Above one can see which Smoothing filters we found to be ideal for the 7 different data files.